In [1]:
# Importing Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Importing Dataset
data = pd.read_csv('USDJPY-2019-01.csv', names=['pair', 'datetime', 'low', 'high'])

In [3]:
data.head(2)

,pair,datetime,low,high
0,USD/JPY,20190101 22:06:11.323,109.630,109.697
1,USD/JPY,20190101 22:06:11.567,109.625,109.697


In [4]:
class Indicators():
    def __init__(self, data, macd_span_1, macd_span_2, rsi_rolling_window, williams_r_rolling_window):
        # Last timestamp
        last_ts = np.array(data[data.datetime<'20190101 22:06:15.567'].datetime.loc[-1:0])[0]
        
        self.data = data
        self.datetime = data.datetime
        self.macd = self.calcMACD(data, macd_span_1, macd_span_2)
        self.rsi = self.calcRSI(data, rsi_rolling_window)
        self.williams_r = self.calcWilliamsR(data, williams_r_rolling_window)
    
    def calcMACD(self, data, macd_span_1, macd_span_2):
        data_span_1_ema = data.ewm(span=macd_span_1).mean()
        data_span_2_ema = data.ewm(span=macd_span_2).mean()

        macd_low = data_span_2_ema.low - data_span_1_ema.low
        macd_high = data_span_2_ema.high - data_span_1_ema.high

        return pd.DataFrame({'pair': data.pair, 'datetime': data.datetime, 'macd_low': macd_low, 'macd_high': macd_high})
    
    def calcRSI(self, data, rsi_rolling_window):
        high = data.high.diff()
        low = data.low.diff()

        high_gain = high.copy()
        high_gain[high_gain< 0] = 0

        high_loss = high.copy()
        high_loss[high_loss> 0] = 0

        low_gain = low.copy()
        low_gain[low_gain< 0] = 0

        low_loss = low.copy()
        low_loss[low_loss> 0] = 0

        gain_loss_ratio_high = high_gain.rolling(rsi_rolling_window).mean()/high_loss.rolling(rsi_rolling_window).mean()
        gain_loss_ratio_low = low_gain.rolling(rsi_rolling_window).mean()/low_loss.rolling(rsi_rolling_window).mean()

        rsi_high = gain_loss_ratio_high.apply(lambda val: 100 - 100/(1+ np.abs(val)))
        rsi_low = gain_loss_ratio_low.apply(lambda val: 100 - 100/(1+ np.abs(val)))

        return pd.DataFrame({'pair': data.pair, 'datetime': data.datetime, 'rsi_low': rsi_low, 'rsi_high': rsi_high})
    
    def calcWilliamsR(self, data, williams_r_rolling_window):
        williams_r_high = ((data.high.rolling(williams_r_rolling_window).max() - data.high)*100)/(data.high.rolling(williams_r_rolling_window).max()- data.high.rolling(williams_r_rolling_window).min())
        williams_r_low = ((data.low.rolling(williams_r_rolling_window).max() - data.high)*100)/(data.low.rolling(williams_r_rolling_window).max()- data.low.rolling(williams_r_rolling_window).min())

        return pd.DataFrame({'pair': data.pair, 'datetime': data.datetime, 'williams_r_low': williams_r_low, 'williams_r_high': williams_r_high})
    
    def returnIndicators(self, timestamp):
        data = self.data
        pair = data.pair[0]
        timestamp = list(data.datetime[(data.datetime <= timestamp)])[-1]
        
        indicators = {
            'pair': pair,
            'datetime': timestamp,
            'low': list(self.data[data.datetime == timestamp].low)[0],
            'high': list(self.data[data.datetime == timestamp].high)[0],
            'macd_low': list(self.macd[data.datetime == timestamp].macd_low)[0],
            'macd_high': list(self.macd[data.datetime == timestamp].macd_high)[0],
            'rsi_low': list(self.rsi[data.datetime == timestamp].rsi_low)[0],
            'rsi_high': list(self.rsi[data.datetime == timestamp].rsi_high)[0],
            'williams_r_low': list(self.williams_r[data.datetime == timestamp].williams_r_low)[0],
            'williams_r_high': list(self.williams_r[data.datetime == timestamp].williams_r_high)[0]
        }
        return indicators

In [5]:
ind = Indicators(data=data, macd_span_1=26, macd_span_2=12, rsi_rolling_window=100, williams_r_rolling_window=14)

In [6]:
ind.returnIndicators('20190101 22:09:01.959')

{'pair': 'USD/JPY',
 'datetime': '20190101 22:09:01.959',
 'low': 109.63,
 'high': 109.695,
 'macd_low': -0.001175854001900234,
 'macd_high': -0.000567599322906176,
 'rsi_low': 50.57471264367829,
 'rsi_high': 47.14285714284931,
 'williams_r_low': -787.4999999988897,
 'williams_r_high': 75.00000000017764}